# NorSWE Large Sample Experiment Tutorial

This notebook demonstrates how to run CONFLUENCE over multiple snow observation sites from the NorSWE dataset.

## Overview

NorSWE provides snow observations (snow water equivalent and snow depth) from a network of stations across the northern hemisphere. Running CONFLUENCE at these sites allows us to:

- Validate snow modeling in Nordic conditions
- Compare model performance across different elevations and climates
- Test model physics in snow-dominated environments
- Analyze mountain snow processes

## 1. Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import yaml
from datetime import datetime
import xarray as xr
import seaborn as sns

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Set up plotting style
plt.style.use('default')
sns.set_palette("coolwarm")
%matplotlib inline

print("Setup complete!")

## 2. Configure the Experiment

In [ ]:
# Configuration for the NorSWE large sample experiment
experiment_config = {
    'experiment_name': 'norswe_tutorial',
    'norswe_path': '/work/comphyd_lab/data/geospatial-data/NorSWE/NorSWE-NorEEN_1979-2021_v2.nc',
    'template_config': '../CONFLUENCE/0_config_files/config_norswe_template.yaml',
    'config_dir': '../CONFLUENCE/0_config_files/norswe',
    'output_dir': './norswe_output',
    'base_path': '/work/comphyd_lab/data/CONFLUENCE_data/norswe',
    'min_completeness': 0.0,  # Minimum % data completeness
    'max_stations': 10,  # Number of stations to process
    'start_year': 2010,  # Optional: filter data by year
    'end_year': 2020,
    'no_submit': False  # Set to True for dry run
}

# Create directories
experiment_dir = Path(f"./experiments/{experiment_config['experiment_name']}")
experiment_dir.mkdir(parents=True, exist_ok=True)
Path(experiment_config['output_dir']).mkdir(parents=True, exist_ok=True)
Path(experiment_config['config_dir']).mkdir(parents=True, exist_ok=True)

# Save configuration
with open(experiment_dir / 'experiment_config.yaml', 'w') as f:
    yaml.dump(experiment_config, f)

print(f"Experiment configured: {experiment_config['experiment_name']}")
print(f"Processing up to {experiment_config['max_stations']} NorSWE stations")

## 3. Explore NorSWE Dataset

In [ ]:
'''
# Open NorSWE dataset
ds = xr.open_dataset(experiment_config['norswe_path'])

print("NorSWE Dataset Information:")
print(f"Time range: {ds.time.values[0]} to {ds.time.values[-1]}")
print(f"Number of stations: {len(ds.station_id)}")
print(f"Variables: {list(ds.data_vars)}")
print(f"Coordinates: {list(ds.coords)}")

# Display dataset structure
print("\nDataset structure:")
print(ds)

ds.close()
'''

## 4. Process NorSWE Station Data

In [ ]:
# Import the processing function from the script
sys.path.append(str(confluence_path / '9_scripts'))
from run_sites_norswe import process_norswe_data

# Process station data
stations_csv = Path('norswe_stations.csv')
stations_df = pd.read_csv(stations_csv)

'''
stations_df = process_norswe_data(
    experiment_config['norswe_path'],
    str(stations_csv),
    start_year=experiment_config.get('start_year'),
    end_year=experiment_config.get('end_year'),
    use_existing_csv=True
)
'''

print(f"Processed {len(stations_df)} stations")
print("\nStation data columns:")
for col in stations_df.columns:
    print(f"  - {col}")

# Display sample stations
print("\nSample stations:")
display(stations_df[['station_id', 'station_name', 'lat', 'lon', 'elevation', 'swq_completeness']].head())


## 5. Visualize Station Distribution

In [ ]:
# Create station distribution map
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

# Geographic distribution
scatter = ax1.scatter(stations_df['lon'], stations_df['lat'], 
                     c=stations_df['elevation'], cmap='terrain',
                     s=50, alpha=0.7, edgecolors='black', linewidth=0.5)
ax1.set_title('NorSWE Station Locations', fontsize=16, fontweight='bold')
ax1.set_xlabel('Longitude', fontsize=12)
ax1.set_ylabel('Latitude', fontsize=12)
ax1.grid(True, alpha=0.3)

# Add colorbar for elevation
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Elevation (m)', fontsize=12)

# Elevation distribution
ax2.hist(stations_df['elevation'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Elevation (m)', fontsize=12)
ax2.set_ylabel('Number of Stations', fontsize=12)
ax2.set_title('Elevation Distribution', fontsize=14)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Data completeness distribution
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(stations_df['swq_completeness'], bins=20, color='lightblue', 
        edgecolor='black', alpha=0.7, label='SWE')
ax.hist(stations_df['snd_completeness'], bins=20, color='lightcoral', 
        edgecolor='black', alpha=0.5, label='Snow Depth')
ax.set_xlabel('Data Completeness (%)', fontsize=12)
ax.set_ylabel('Number of Stations', fontsize=12)
ax.set_title('Data Completeness Distribution', fontsize=14)
ax.legend()
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Select Stations for Processing

In [ ]:
# Filter stations by data completeness
complete_stations = stations_df[
    (stations_df['swq_completeness'] >= experiment_config['min_completeness']) &
    (stations_df['snd_completeness'] >= experiment_config['min_completeness'])
].copy()

print(f"Found {len(complete_stations)} stations with ≥{experiment_config['min_completeness']}% completeness")

# Select stations (prioritize by completeness and elevation diversity)
if len(complete_stations) > experiment_config['max_stations']:
    # Sort by completeness and select diverse elevations
    complete_stations = complete_stations.sort_values(
        by=['swq_completeness', 'snd_completeness'], 
        ascending=False
    ).head(experiment_config['max_stations'])

print(f"\nSelected {len(complete_stations)} stations for processing")
display(complete_stations[['station_id', 'station_name', 'elevation', 'swq_completeness']])

# Save selected stations
complete_stations.to_csv(experiment_dir / 'selected_stations.csv', index=False)

## 7. Extract Snow Observation Data

In [ ]:
'''
# Example: Extract and plot snow data for one station
from run_sites_norswe import extract_snow_data

# Select first station as example
example_station = complete_stations.iloc[0]
station_id = example_station['station_id']
station_name = example_station['Watershed_Name']

# Create output directory for this station
station_dir = Path(experiment_config['base_path']) / f"domain_{station_name}"
station_dir.mkdir(parents=True, exist_ok=True)

# Extract snow data
swe_file, snd_file = extract_snow_data(
    experiment_config['norswe_path'],
    station_id,
    str(station_dir),
    start_year=experiment_config.get('start_year'),
    end_year=experiment_config.get('end_year')
)

# Load and visualize the extracted data
swe_df = pd.read_csv(swe_file)
snd_df = pd.read_csv(snd_file)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Plot SWE
ax1.plot(pd.to_datetime(swe_df['time']), swe_df['SWE_kg_m2'], 
         color='blue', linewidth=1.5, alpha=0.7)
ax1.set_ylabel('SWE (kg/m²)', fontsize=12)
ax1.set_title(f'Snow Observations - {station_name} (ID: {station_id})', fontsize=14)
ax1.grid(True, alpha=0.3)

# Plot snow depth
ax2.plot(pd.to_datetime(snd_df['time']), snd_df['Depth_m'] * 100, 
         color='purple', linewidth=1.5, alpha=0.7)
ax2.set_ylabel('Snow Depth (cm)', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Extracted snow data for {station_name}:")
print(f"  SWE file: {swe_file}")
print(f"  Snow depth file: {snd_file}")
'''

## 8. Generate Configuration Files

In [ ]:
'''
# Generate configs for selected stations
from run_sites_norswe import generate_config_file

config_dir = Path(experiment_config['config_dir'])
generated_configs = []

for _, station in complete_stations.iterrows():
    station_name = station['Watershed_Name']
    pour_point = station['POUR_POINT_COORDS']
    bounding_box = station['BOUNDING_BOX_COORDS']
    
    # Generate config file
    config_path = config_dir / f"config_{station_name}.yaml"
    
    generate_config_file(
        experiment_config['template_config'],
        str(config_path),
        station_name,
        pour_point,
        bounding_box
    )
    
    generated_configs.append(config_path)

print(f"Generated {len(generated_configs)} configuration files")
print("\nExample config locations:")
for config in generated_configs[:3]:
    print(f"  - {config}")
'''

## 9. Launch CONFLUENCE Jobs

In [ ]:
'''
# Prepare to launch CONFLUENCE runs
from run_sites_norswe import run_confluence

submitted_jobs = []
skipped_jobs = []

# Interactive decision (for notebook, we'll simulate 'y' or 'n')
if experiment_config['no_submit']:
    submit_jobs = 'n'
    print("DRY RUN MODE - No jobs will be submitted")
else:
    submit_jobs = 'y'  # In real notebook, you'd ask user
    print("Preparing to submit CONFLUENCE jobs...")

if submit_jobs == 'y':
    for _, station in complete_stations.iterrows():
        station_name = station['Watershed_Name']
        
        # Check if simulation already exists
        sim_path = Path(experiment_config['base_path']) / f"domain_{station_name}" / "simulations" / "run_1" / "SUMMA" / "run_1_timestep.nc"
        
        if sim_path.exists():
            print(f"Skipping {station_name} - simulation already exists")
            skipped_jobs.append(station_name)
            continue
        
        # Submit job
        config_path = config_dir / f"config_{station_name}.yaml"
        job_id = run_confluence(str(config_path), station_name)
        
        if job_id:
            submitted_jobs.append((station_name, job_id))
            print(f"Submitted job for {station_name}: {job_id}")
        
        # Small delay between submissions
        import time
        time.sleep(2)

# Summary
print("\nJob submission summary:")
print(f"  Submitted: {len(submitted_jobs)}")
print(f"  Skipped: {len(skipped_jobs)}")

if submitted_jobs:
    print("\nSubmitted jobs:")
    for station_name, job_id in submitted_jobs[:5]:
        print(f"  {station_name}: {job_id}")
'''

## 10. Monitor Job Status

In [ ]:
'''
# Check job status
def check_job_status(user=None):
    user = user or os.environ.get('USER')
    cmd = ['squeue', '-u', user]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout

print("Current job status:")
print(check_job_status())

# Save job information
if submitted_jobs:
    job_df = pd.DataFrame(submitted_jobs, columns=['station_name', 'job_id'])
    job_df.to_csv(experiment_dir / 'submitted_jobs.csv', index=False)
    print(f"\nJob information saved to {experiment_dir / 'submitted_jobs.csv'}")
'''

## 11. Find Completed Simulations

In [ ]:
# Find completed simulations
base_path = Path(experiment_config['base_path'])
completed = []

for _, station in complete_stations.iterrows():
    station_name = station['Watershed_Name']
    sim_path = base_path / f"domain_{station_name}" / "simulations" / "run_1" / "SUMMA"
    
    if sim_path.exists() and list(sim_path.glob("*timestep*.nc")):
        completed.append({
            'station_name': station_name,
            'station_id': station['station_id'],
            'elevation': station['elevation'],
            'sim_path': sim_path
        })

print(f"Found {len(completed)} completed simulations")
if completed:
    completed_df = pd.DataFrame(completed)
    display(completed_df[['station_name', 'station_id', 'elevation']])

## 12. Load and Compare Results

In [ ]:
# Function to load SUMMA snow output
def load_summa_snow(sim_path):
    output_files = list(Path(sim_path).glob("*timestep*.nc"))
    if output_files:
        ds = xr.open_dataset(output_files[0])
        
        # Extract SWE and snow depth if available
        data = {}
        if 'scalarSWE' in ds.variables:
            data['swe'] = ds.scalarSWE.values.flatten()
        if 'scalarSnowDepth' in ds.variables:
            data['depth'] = ds.scalarSnowDepth.values.flatten()
        
        data['time'] = pd.to_datetime(ds.time.values)
        ds.close()
        
        return pd.DataFrame(data)
    return None

# Compare modeled and observed snow for completed simulations
if completed:
    fig, axes = plt.subplots(len(completed[:3]), 2, figsize=(15, 4*len(completed[:3])))
    
    for i, site in enumerate(completed[:3]):
        # Load model output
        model_data = load_summa_snow(site['sim_path'])
        
        # Load observations
        obs_path = base_path / f"domain_{site['station_name']}" / "observations" / "snow" / "raw_data"
        swe_obs = pd.read_csv(obs_path / "swe" / f"{site['station_id']}_swe.csv")
        snd_obs = pd.read_csv(obs_path / "depth" / f"{site['station_id']}_depth.csv")
        
        if model_data is not None:
            # Plot SWE
            ax1 = axes[i, 0] if len(completed) > 1 else axes[0]
            ax1.plot(pd.to_datetime(swe_obs['time']), swe_obs['SWE_kg_m2'], 
                    'k-', label='Observed', linewidth=2)
            if 'swe' in model_data:
                ax1.plot(model_data['time'], model_data['swe'], 
                        'r-', label='Modeled', alpha=0.7)
            ax1.set_ylabel('SWE (kg/m²)')
            ax1.set_title(f"{site['station_name']} - Elevation: {site['elevation']}m")
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Plot snow depth
            ax2 = axes[i, 1] if len(completed) > 1 else axes[1]
            ax2.plot(pd.to_datetime(snd_obs['time']), snd_obs['Depth_m'] * 100, 
                    'k-', label='Observed', linewidth=2)
            if 'depth' in model_data:
                ax2.plot(model_data['time'], model_data['depth'] * 100, 
                        'r-', label='Modeled', alpha=0.7)
            ax2.set_ylabel('Snow Depth (cm)')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 13. Performance Analysis Across Elevations